## Optimize the Model


In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import requests
import io

#  Import and read the charity_data.csv.
import pandas as pd 
url = "https://raw.githubusercontent.com/kawandag/deep_learning-challenge/main/charity_data.csv"
r = requests.get(url).content



In [2]:
application_df = pd.read_csv(io.StringIO(r.decode('utf-8')),low_memory=False)
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Drop the non-beneficial EIN column
new_app = application_df.drop(columns={'EIN'})
new_app

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [7]:
# display the number of unique values

new_app.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [16]:
# Look at NAME value counts for binning
name_df = new_app["NAME"].value_counts()

#  How many name counts are greater than 5?
name_df[name_df > 5]
name_df

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64

In [17]:
# Determine which values to replace if counts are less than or equal to 5.
values_replace = list(name_df[name_df <= 5].index)

# Replace in dataframe
for v in values_replace:
    new_app['NAME'] = new_app['NAME'].replace(v,"Other")

    
# Check to make sure binning was successful
new_app['NAME'].value_counts()

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64

In [41]:
# Look at APPLICATION_TYPE value counts for binning
app_type = new_app["APPLICATION_TYPE"].value_counts()
app_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [42]:
# Determine which values to replace if application counts are less than 500
app_values_r = list(app_type[app_type < 500])

# Replace in dataframe
for app in app_values_r:
  new_app['APPLICATION_TYPE'] = new_app['APPLICATION_TYPE'].replace(app,"Other")

   


# Check to make sure binning was successful
new_app["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [43]:
# Look at CLASSIFICATION value counts for binning
class_it = new_app["CLASSIFICATION"].value_counts()
class_it

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [46]:
# Determine which values to replace if counts are less than 1000
class_replace = list(class_it[class_it < 1000].index)

# Replace in dataframe
for c in class_replace:
  new_app["CLASSIFICATION"] = new_app["CLASSIFICATION"].replace(c,"Other")
    
# Check to make sure binning was successful
new_app["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [52]:
# Generate our categorical variable lists
new_app_dummies = pd.get_dummies(new_app)
new_app_dummies.dtypes

STATUS                                    int64
ASK_AMT                                   int64
IS_SUCCESSFUL                             int64
NAME_AACE INTERNATIONAL                   uint8
NAME_ACE MENTOR PROGRAM OF AMERICA INC    uint8
                                          ...  
INCOME_AMT_25000-99999                    uint8
INCOME_AMT_50M+                           uint8
INCOME_AMT_5M-10M                         uint8
SPECIAL_CONSIDERATIONS_N                  uint8
SPECIAL_CONSIDERATIONS_Y                  uint8
Length: 407, dtype: object

In [54]:
# perform one-hot encoding on the data

new_app_dummies = pd.get_dummies(new_app)
new_app_dummies

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [55]:
# Split our preprocessed data into our features and target arrays
X = new_app_dummies.drop(["IS_SUCCESSFUL"], axis='columns').values
y = new_app_dummies["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(X,y)

In [56]:
# Create a StandardScaler instances
scaler = StandardScaler()


# Fit the StandardScaler
X_scaler = scaler.fit(x_train)

# Scale the data
X_train_scaled = X_scaler.transform(x_train)
X_test_scaled = X_scaler.transform(x_test)

In [58]:
# Define the model - deep neural net
number_input_features = len(x_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               40700     
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 44,051
Trainable params: 44,051
Non-trainable params: 0
_________________________________________________________________


In [60]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [61]:
# Train the model
model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.4967 - accuracy: 0.7726
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4420 - accuracy: 0.7945
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4338 - accuracy: 0.7965
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4295 - accuracy: 0.7966
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4270 - accuracy: 0.7965
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4244 - accuracy: 0.7975
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4230 - accuracy: 0.7993
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4220 - accuracy: 0.7992
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4206 - accuracy: 0.7997
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4196 - accuracy: 0.7996

In [62]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4472 - accuracy: 0.7896 - 523ms/epoch - 2ms/step
Loss: 0.44722652435302734, Accuracy: 0.7896209955215454


In [63]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

USE A RANDOM FOREST CLASSIFIER ON THE DATASET

In [64]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [69]:
from scipy.sparse.construct import random
# Create a random forest classifier.
cls = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
cls = cls.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = cls.predict(X_test_scaled)
print(f"Random forest model accuracy:{accuracy_score(y_test, y_pred):.3f}")

Random forest model accuracy:0.776
